In [1]:
import pandas as pd
import os, dill
import numpy as np
import cvxpy as cp
from typing import Iterable, Optional, Mapping, cast
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from ecoli.processes.metabolism_redux_classic import NetworkFlowModel, FlowResult
os.chdir(os.path.expanduser('~/dev/vEcoli'))

%load_ext autoreload

In [2]:
# import sim
time = '600'
date = '2026-01-22'
experiment = 'homeostatic_only'
condition = 'basal'
entry = f'{experiment}_{time}_{date}'
folder = f'out/objective_weight/{condition}/{entry}/'

output = np.load(folder + '0_output.npy',allow_pickle='TRUE').item()
output = output['agents']['0']
fba = output['listeners']['fba_results']
bulk = pd.DataFrame(output['bulk'])
f = open(folder + 'agent_steps.pkl', 'rb')
agent = dill.load(f)
f.close()

metabolism = agent['ecoli-metabolism-redux-classic']

In [3]:
# get objective terms across time
counts_to_molar = output['listeners']['enzyme_kinetics']['counts_to_molar']

homeostatic_term = np.asarray(fba['homeostatic_term'])/counts_to_molar #change to count
# secretion_term = np.asarray(fba['secretion_term'])/counts_to_molar
# efficiency_term = np.asarray(fba['efficiency_term'])/counts_to_molar
# kinetic_term = np.asarray(fba['kinetic_term'])/counts_to_molar

time = np.arange(len(homeostatic_term))

objective_weights_sim = {
            "secretion": 0.01,
            "efficiency": 0.001,
            "kinetics": 0.005,
            "homeostatic": 1,
}

In [4]:
from plotly.colors import qualitative as q

spec_term = {
    "Homeostatic": (homeostatic_term, objective_weights_sim['homeostatic']),
    # "Secretion":   (secretion_term,   objective_weights_sim['secretion']),
    # "Efficiency":  (efficiency_term,  objective_weights_sim['efficiency']),
    # "Kinetic":     (kinetic_term,     objective_weights_sim['kinetics']),
}

spec_no_weights = {
    "Homeostatic": (homeostatic_term/objective_weights_sim['homeostatic'], objective_weights_sim['homeostatic']),
    # "Secretion":   (secretion_term/objective_weights_sim['secretion'],     objective_weights_sim['secretion']),
    # "Efficiency":  (efficiency_term/objective_weights_sim['efficiency'],   objective_weights_sim['efficiency']),
    # "Kinetic":     (kinetic_term/objective_weights_sim['kinetics'],        objective_weights_sim['kinetics']),
}


palette = q.Plotly  # consistent colors across both sets
colors = {name: palette[i % len(palette)] for i, name in enumerate(spec_term)}


fig = go.Figure()

# Add weighted (solid) + unweighted (dotted) per term
for i, (name, (y_w, w)) in enumerate(spec_term.items()):
    # solid, weighted
    fig.add_trace(go.Scatter(
        x=time, y=y_w, mode="lines",
        name=f"{name} (weighted)",
        line=dict(color=colors[name], dash="solid"),
        legendgroup=name,
        legendgrouptitle_text=name if i == 0 else None,
        hovertemplate=f"{name.lower()} weight={w}<br>{name.lower()} term value=%{{y:.4g}}<extra></extra>"
    ))
    # dotted, unweighted
    y_uw, w_uw = spec_no_weights[name]
    fig.add_trace(go.Scatter(
        x=time, y=y_uw, mode="lines",
        name=f"{name} (unweighted)",
        line=dict(color=colors[name], dash="dot"),
        legendgroup=name,
        hovertemplate=f"{name.lower()} (no weight)<br>{name.lower()} value=%{{y:.4g}}<extra></extra>"
    ))

fig.update_layout(
    title="Change of Objective Term Values Over Sim",
    xaxis_title="Time (s)",
    yaxis=dict(title="Objective Term Value", type="log", tickformat=".0e"),
    template="plotly_white",
    legend=dict(title="Terms", groupclick="togglegroup"),  # click once to toggle both lines in a group
    width=1200, height=600,
)

fig.show()
# fig.write_image("/Users/HeenaSaqib/dev/vEcoli/notebooks/Heena notebooks/Metabolism_New Genes/out/objective_weights/change of objective term values over a sim.png", scale=3)

In [5]:
# find top 10 homeostatic loss
estimated_homeostatic_dmdt = pd.DataFrame(fba["estimated_homeostatic_dmdt"][1:], columns=metabolism.homeostatic_metabolites)
target_homeostatic_dmdt = pd.DataFrame(fba["target_homeostatic_dmdt"][1:], columns=metabolism.homeostatic_metabolites)
homeostatic_count = pd.DataFrame(np.asarray(fba["homeostatic_metabolite_counts"][1:]), columns = metabolism.homeostatic_metabolites)

# calculate individual metabolite objective loss
homeostatic_objective  = np.abs(target_homeostatic_dmdt - estimated_homeostatic_dmdt)/homeostatic_count

# find top 10 metabolites with most lost over time
met_loss_avg = homeostatic_objective.mean(axis=0)
time_loss_avg = homeostatic_objective.mean(axis=1)

temp = homeostatic_objective[time_loss_avg > 0]
temp = temp.loc[:, met_loss_avg > 0]
temp

,CPD-12261[p],glycogen-monomer[c]
0,0.000000,4.923154e-08
62,0.000269,0.000000e+00
63,0.000533,0.000000e+00
64,0.000796,0.000000e+00
65,0.001054,0.000000e+00
...,...,...
595,0.014069,0.000000e+00
596,0.014347,0.000000e+00
597,0.014625,0.000000e+00
598,0.014903,0.000000e+00


In [6]:
def get_subset_S(S, met_of_interest):
    S_met = S.loc[met_of_interest, :]
    S_met = S_met.loc[:,~np.all(S_met == 0, axis=0)]
    return S_met, S_met.columns

In [7]:
stoichiometry = metabolism.stoichiometry.copy()
reaction_names = metabolism.reaction_names
metabolites = metabolism.metabolite_names.copy()
S = stoichiometry .copy()
S = pd.DataFrame(S, index=metabolites , columns=reaction_names)


S_met, rxns =  get_subset_S(S, temp.columns)
S_met

,RXN-11302,glycogen-monomer-extension
CPD-12261[p],1,0
glycogen-monomer[c],0,1


## These metabolites are both manually added for murein (CPD-12261) and glycogen. It's odd to see that flow can be sustained through these metabolites at some time steps but the need is n't met for some other timestep, even when homeosatic objective is the only objective present.

In [8]:
###----- 1. Visualize metabolite exchange -----###
met_of_interest = ["CPD-12261[p]", "glycogen-monomer[c]"]

dmdt_diff = (target_homeostatic_dmdt - estimated_homeostatic_dmdt)/homeostatic_count
dmdt_diff_interest = dmdt_diff.loc[:,met_of_interest]
dmdt_diff_interest

,CPD-12261[p],glycogen-monomer[c]
0,0.000000,-4.923154e-08
1,0.000000,0.000000e+00
2,0.000000,0.000000e+00
3,0.000000,0.000000e+00
4,0.000000,0.000000e+00
...,...,...
595,0.014069,0.000000e+00
596,0.014347,0.000000e+00
597,0.014625,0.000000e+00
598,0.014903,0.000000e+00


In [9]:
import plotly.express as px

# dmdt_diff_interest: rows = timepoints, columns = metabolites
df = dmdt_diff_interest.copy()

# Make sure the index is your time axis (seconds)
df = df.copy()
df.index.name = "time_s"

# Wide -> long
long = (
    df.reset_index()
      .melt(id_vars="time_s", var_name="metabolite", value_name="unmet_need")
)

fig = px.line(
    long,
    x="time_s",
    y="unmet_need",
    color="metabolite",
    markers=False,  # set True if you want dots
    title="Normalized dmdt difference over time",
)
fig.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="Normalized (Target - Estimate)",
    legend_title="Metabolite",
    template="plotly_white",
    # paper_bgcolor='rgba(255, 0, 0, 0)',
    # plot_bgcolor='rgba(255, 0, 0, 0)'
)
fig.show()

### glycogen-monomer[c] was made too much at timestep 1 (because diff is negative --> estimate > target). CPD-12261[p] is the main culprit for unmet need

In [10]:
###----- 2. Check if any of the reactions upstream of these metabolites have no enzyme, causing zero flow-----###
# 2.1 Run parsimonious FBA (pFBA) with to maximize CPD-12261 export while minimizing total flux to extract path of making CPD-12261
import cvxpy as cp
v = cp.Variable(len(reaction_names))
e = cp.Variable(NetworkFlowModel.n_exch_rxns)
dm = NetworkFlowModel.S_orig @ v + NetworkFlowModel.S_exch @ e
exch = NetworkFlowModel.S_exch @ e
upper_flux_bound = 100

constr = []
constr.append(dm[NetworkFlowModel.intermediates_idx] == 0)
constr.extend([v >= 0, v <= upper_flux_bound, e >= 0, e <= upper_flux_bound])

# objective is to maximize dm[index_CPD-12261[p]] and minimize total flux
CPD_12261_index = metabolites.index('CPD-12261[p]')
objective = -dm[CPD_12261_index] + cp.sum(v)

problem = cp.Problem(cp.Minimize(objective), constr)



AttributeError: type object 'NetworkFlowModel' has no attribute 'n_exch_rxns'

In [71]:
NetworkFlowModel.n_exch_rxns

AttributeError: type object 'NetworkFlowModel' has no attribute 'n_exch_rxns'

973